Reference: https://www.sbert.net/examples/applications/retrieve_rerank/README.html

In [1]:
import pandas as pd
from nltk import tokenize, sent_tokenize
import nltk
nltk.download('punkt')
import time

[nltk_data] Downloading package punkt to /Users/dylantan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# df = pd.read_pickle('data.df')
df =pd.read_csv('sample_text.csv')

In [3]:
df

,title,context
0,Symbiosis,"Symbiosis (from Greek σύν ""together"" and βίωσι..."
1,Samoa,The Independent State of Samoa ( Samoan: Malo ...
2,House_music,House music is a genre of electronic dance mus...
3,Czech_language,Czech (/ˈtʃɛk/; čeština Czech pronunciation: [...
4,Dialect,"The term dialect (from Latin dialectus, dialec..."
5,Hanover,"In 1636 George, Duke of Brunswick-Lüneburg, ru..."
6,Digestion,"In the human digestive system, food enters the..."
7,IBM,The company originated in 1911 as the Computin...
8,Neolithic,Traditionally considered the last part of the ...
9,Sexual_orientation,Scientists do not know the exact cause of sexu...


In [4]:
# %pip install -U sentence-transformers rank_bm25

In [5]:
# para = """Neural networks, also known as artificial neural networks (ANNs) or simulated neural networks (SNNs), are a subset of machine learning and are at the heart of deep learning algorithms. Their name and structure are inspired by the human brain, mimicking the way that biological neurons signal to one another.

# Artificial neural networks (ANNs) are comprised of a node layers, containing an input layer, one or more hidden layers, and an output layer. Each node, or artificial neuron, connects to another and has an associated weight and threshold. If the output of any individual node is above the specified threshold value, that node is activated, sending data to the next layer of the network. Otherwise, no data is passed along to the next layer of the network.

# Neural networks rely on training data to learn and improve their accuracy over time. However, once these learning algorithms are fine-tuned for accuracy, they are powerful tools in computer science and artificial intelligence, allowing us to classify and cluster data at a high velocity. Tasks in speech recognition or image recognition can take minutes versus hours when compared to the manual identification by human experts. One of the most well-known neural networks is Google’s search algorithm.

# Once an input layer is determined, weights are assigned. These weights help determine the importance of any given variable, with larger ones contributing more significantly to the output compared to other inputs. All inputs are then multiplied by their respective weights and then summed. Afterward, the output is passed through an activation function, which determines the output. If that output exceeds a given threshold, it “fires” (or activates) the node, passing data to the next layer in the network. This results in the output of one node becoming in the input of the next node. This process of passing data from one layer to the next layer defines this neural network as a feedforward network.

# If we use the activation function from the beginning of this section, we can determine that the output of this node would be 1, since 6 is greater than 0. In this instance, you would go surfing; but if we adjust the weights or the threshold, we can achieve different outcomes from the model. When we observe one decision, like in the above example, we can see how a neural network could make increasingly complex decisions depending on the output of previous decisions or layers.

# In the example above, we used perceptrons to illustrate some of the mathematics at play here, but neural networks leverage sigmoid neurons, which are distinguished by having values between 0 and 1. Since neural networks behave similarly to decision trees, cascading data from one node to another, having x values between 0 and 1 will reduce the impact of any given change of a single variable on the output of any given node, and subsequently, the output of the neural network.

# Neural networks can be classified into different types, which are used for different purposes. While this isn’t a comprehensive list of types, the below would be representative of the most common types of neural networks that you’ll come across for its common use cases:

# The perceptron is the oldest neural network, created by Frank Rosenblatt in 1958.

# Feedforward neural networks, or multi-layer perceptrons (MLPs), are what we’ve primarily been focusing on within this article. They are comprised of an input layer, a hidden layer or layers, and an output layer. While these neural networks are also commonly referred to as MLPs, it’s important to note that they are actually comprised of sigmoid neurons, not perceptrons, as most real-world problems are nonlinear. Data usually is fed into these models to train them, and they are the foundation for computer vision, natural language processing, and other neural networks.

# Convolutional neural networks (CNNs) are similar to feedforward networks, but they’re usually utilized for image recognition, pattern recognition, and/or computer vision. These networks harness principles from linear algebra, particularly matrix multiplication, to identify patterns within an image.

# Recurrent neural networks (RNNs) are identified by their feedback loops. These learning algorithms are primarily leveraged when using time-series data to make predictions about future outcomes, such as stock market predictions or sales forecasting.

# Deep Learning and neural networks tend to be used interchangeably in conversation, which can be confusing. As a result, it’s worth noting that the “deep” in deep learning is just referring to the depth of layers in a neural network. A neural network that consists of more than three layers—which would be inclusive of the inputs and the output—can be considered a deep learning algorithm. A neural network that only has two or three layers is just a basic neural network.

# """
# title = 'what are neural networks'
# print(title)

In [55]:
idx = 4
para = df['context'][idx]
title = df['title'][idx]
print(f"Title: {title}")
print(f"Para: {para}")


Title: Dialect
Para: The term dialect (from Latin dialectus, dialectos, from the ancient Greek word διάλεκτος diálektos, "discourse", from διά diá, "through" and λέγω legō, "I speak") is used in two distinct ways to refer to two different types of linguistic phenomena. One usage—the more common among linguists—refers to a variety of a language that is a characteristic of a particular group of the language's speakers. The term is applied most often to regional speech patterns, but a dialect may also be defined by other factors, such as social class. A dialect that is associated with a particular social class can be termed a sociolect, a dialect that is associated with a particular ethnic group can be termed as ethnolect, and a regional dialect may be termed a regiolect. According to this definition, any variety of a language constitutes "a dialect", including any standard varieties. The other usage refers to a language that is socially subordinated to a regional or national standard lan

In [56]:
passage = sent_tokenize(para)
print(len(passage))
passage

150


['The term dialect (from Latin dialectus, dialectos, from the ancient Greek word διάλεκτος diálektos, "discourse", from διά diá, "through" and λέγω legō, "I speak") is used in two distinct ways to refer to two different types of linguistic phenomena.',
 "One usage—the more common among linguists—refers to a variety of a language that is a characteristic of a particular group of the language's speakers.",
 'The term is applied most often to regional speech patterns, but a dialect may also be defined by other factors, such as social class.',
 'A dialect that is associated with a particular social class can be termed a sociolect, a dialect that is associated with a particular ethnic group can be termed as ethnolect, and a regional dialect may be termed a regiolect.',
 'According to this definition, any variety of a language constitutes "a dialect", including any standard varieties.',
 'The other usage refers to a language that is socially subordinated to a regional or national standard la

In [57]:
# import json
# from sentence_transformers import SentenceTransformer, CrossEncoder, util
# import gzip
# import os
# import torch

# if not torch.cuda.is_available():
#     print("Warning: No GPU found. Please add GPU to your notebook")


# #We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
# bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
# bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
# top_k = 32                          #Number of passages we want to retrieve with the bi-encoder

# #The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
# cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# # As dataset, we use Simple English Wikipedia. Compared to the full English wikipedia, it has only
# # about 170k articles. We split these articles into paragraphs and encode them with the bi-encoder

# wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

# if not os.path.exists(wikipedia_filepath):
#     util.http_get('http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz', wikipedia_filepath)

# passages = []
# with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
#     for line in fIn:
#         data = json.loads(line.strip())

#         #Add all paragraphs
#         #passages.extend(data['paragraphs'])

#         #Only add the first paragraph
#         passages.append(data['paragraphs'][0])

# print("Passages:", len(passages))

# # We encode all passages into our vector space. This takes about 5 minutes (depends on your GPU speed)
# corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

In [58]:
# print(passages[:100])

A critical distinction for your setup is symmetric vs. asymmetric semantic search:

For **symmetric** semantic search your **query and the entries in your corpus are of about the same length** and have the same amount of content. An example would be searching for similar questions: Your query could for example be “How to learn Python online?” and you want to find an entry like “How to learn Python on the web?”. For symmetric tasks, you could potentially flip the query and the entries in your corpus.

For **asymmetric** semantic search, you usually have a** short query** (like a question or some keywords) and you want to **find a longer paragraph** answering the query. An example would be a query like “What is Python” and you wand to find the paragraph “Python is an interpreted, high-level and general-purpose programming language. Python’s design philosophy …”. For asymmetric tasks, flipping the query and the entries in your corpus usually does not make sense.

In [59]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")


#We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
bi_encoder = SentenceTransformer('msmarco-distilbert-base-v4')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
top_k = 10                          #Number of passages we want to retrieve with the bi-encoder

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# We encode all passages into our vector space. This takes about 5 minutes (depends on your GPU speed)
corpus_embeddings = bi_encoder.encode(passage, convert_to_tensor=True, show_progress_bar=True)

Batches: 100%|██████████| 5/5 [00:04<00:00,  1.08it/s]


In [60]:
# We also compare the results to lexical search (keyword search). Here, we use 
# the BM25 algorithm which is implemented in the rank_bm25 package.

from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np


# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc


tokenized_corpus = []
for psg in tqdm(passage):
    tokenized_corpus.append(bm25_tokenizer(psg))

bm25 = BM25Okapi(tokenized_corpus)

100%|██████████| 150/150 [00:00<00:00, 32418.49it/s]


In [61]:
print(tokenized_corpus)

[['term', 'dialect', 'latin', 'dialectus', 'dialectos', 'ancient', 'greek', 'word', 'διάλεκτος', 'diálektos', 'discourse', 'διά', 'diá', 'λέγω', 'legō', 'speak', 'used', 'distinct', 'ways', 'refer', 'different', 'types', 'linguistic', 'phenomena'], ['usage—the', 'common', 'linguists—refers', 'variety', 'language', 'characteristic', 'particular', 'group', "language's", 'speakers'], ['term', 'applied', 'regional', 'speech', 'patterns', 'dialect', 'defined', 'factors', 'social', 'class'], ['dialect', 'associated', 'particular', 'social', 'class', 'termed', 'sociolect', 'dialect', 'associated', 'particular', 'ethnic', 'group', 'termed', 'ethnolect', 'regional', 'dialect', 'termed', 'regiolect'], ['according', 'definition', 'variety', 'language', 'constitutes', 'dialect', 'including', 'standard', 'varieties'], ['usage', 'refers', 'language', 'socially', 'subordinated', 'regional', 'national', 'standard', 'language', 'historically', 'cognate', 'related', 'standard', 'language', 'actually', '

In [62]:
# This function will search all wikipedia articles for passage that
# answer the query

n = 20
cross_n = 10
def bm25search(query):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -n)[-n:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
    
    print(f"Top-{n} lexical search (BM25) hits")
    for hit in bm25_hits[0:n]:
        print("\t{:.3f}\t{}".format(hit['score'], passage[hit['corpus_id']]))

    ##### Re-Ranking #####
    # Now, score all retrieved passage with the cross_encoder
    cross_inp = [[query, passage[hit['corpus_id']]] for hit in bm25_hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        bm25_hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from re-ranker (BM25)
    print("\n-------------------------\n")
    print(f"Top-{top_k} Cross-Encoder Re-ranker hits")
    bm25_hits = sorted(bm25_hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in bm25_hits[0:cross_n]:
        print("\t{:.3f}\t{}".format(hit['cross-score'], passage[hit['corpus_id']].replace("\n", " ")))


In [63]:
bm25search(query = title)

Input question: Dialect
Top-20 lexical search (BM25) hits
	1.872	A standard dialect (also known as a standardized dialect or "standard language") is a dialect that is supported by institutions.
	1.819	However, this definition becomes problematic in the case of dialect continua, in which it may be the case that dialect B is mutually intelligible with both dialect A and dialect C but dialects A and C are not mutually intelligible with each other.
	1.628	A dialect that is associated with a particular social class can be termed a sociolect, a dialect that is associated with a particular ethnic group can be termed as ethnolect, and a regional dialect may be termed a regiolect.
	1.604	By the definition most commonly used by linguists, any linguistic variety can be considered a "dialect" of some language—"everybody speaks a dialect".
	1.530	The terms "language" and "dialect" are not necessarily mutually exclusive: There is nothing contradictory in the statement "the language of the Pennsylvan

In [64]:
# This function will search all wikipedia articles for passage that
# answer the query
def semanticsearch(query):
    print("Input question:", query)

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passage
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=n)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passage with the cross_encoder
    cross_inp = [[query, passage[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    print("\n-------------------------\n")
    print(f"Top-{n} Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:n]:
        print("\t{:.3f}\t{}".format(hit['score'], passage[hit['corpus_id']].replace("\n", " ")))

    # Output of top-5 hits from re-ranker (Semantic search)
    print("\n-------------------------\n")
    print("Top-10 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['cross-score'], passage[hit['corpus_id']].replace("\n", " ")))

In [65]:
semanticsearch(query = title)

Input question: Dialect

-------------------------

Top-20 Bi-Encoder Retrieval hits
	0.693	In this sense, unlike in the first usage, the standard language would not itself be considered a "dialect," as it is the dominant language in a particular state or region, whether in terms of social or political status, official status, predominance or prevalence, or all of the above.
	0.691	The term dialect (from Latin dialectus, dialectos, from the ancient Greek word διάλεκτος diálektos, "discourse", from διά diá, "through" and λέγω legō, "I speak") is used in two distinct ways to refer to two different types of linguistic phenomena.
	0.661	A standard dialect (also known as a standardized dialect or "standard language") is a dialect that is supported by institutions.
	0.660	A dialect is distinguished by its vocabulary, grammar, and pronunciation (phonology, including prosody).
	0.636	In many societies, however, a particular dialect, often the sociolect of the elite class, comes to be identifie